PREDICTING APPARTMENT PRICES IN BELGIUM

In [354]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

*COLUMNS TO USE* IMPORT CLEANED DATASET

In [355]:

columns_to_use=['transaction.sale.price','flags.isNewlyBuilt','property.subtype','property.bedroomCount','property.bathroomCount','property.netHabitableSurface','property.hasBasement','property.hasDressingRoom','property.building.condition','property.hasCaretakerOrConcierge','property.hasDisabledAccess','property.hasLift','property.constructionPermit.hasPlotDivisionAuthorization','property.constructionPermit.hasPossiblePriorityPurchaseRight','property.kitchen.type','property.land.hasPlotToRear','property.land.isFlat','property.land.isWooded','property.hasLaundryRoom','property.hasGarden','property.parkingCountIndoor','property.parkingCountOutdoor','property.hasAirConditioning','property.hasArmoredDoor','property.hasVisiophone','property.hasSecureAccessAlarm','property.hasCableTV','property.hasDoorPhone','property.hasSwimmingPool','property.hasSauna','property.hasJacuzzi','property.fireplaceExists','property.hasTerrace','transaction.sale.isSubjectToVat','transaction.certificates.epcScore','property.energy.hasHeatPump','property.energy.hasPhotovoltaicPanels','property.energy.hasThermicPanels','property.energy.hasCollectiveWaterHeater','property.energy.hasDoubleGlazing','transaction.investor.isInvestmentProperty','property.livingRoom.surface'


                ]
FILE='03_cleaned.csv'

df=pd.read_csv(FILE,low_memory=False,usecols=columns_to_use) 
df.shape

(25826, 42)

*BOOL & CATEGORY & NUM & ORDINAL FEATURES *

In [356]:
bool_features=['flags.isNewlyBuilt','property.hasBasement','property.hasDressingRoom','property.hasCaretakerOrConcierge','property.hasDisabledAccess','property.hasLift','property.constructionPermit.hasPlotDivisionAuthorization','property.constructionPermit.hasPossiblePriorityPurchaseRight','property.land.hasPlotToRear','property.land.isFlat','property.land.isWooded','property.hasLaundryRoom','property.hasGarden','property.parkingCountIndoor','property.parkingCountOutdoor','property.hasAirConditioning','property.hasArmoredDoor','property.hasVisiophone','property.hasSecureAccessAlarm','property.hasCableTV','property.hasDoorPhone','property.hasSwimmingPool','property.hasSauna','property.hasJacuzzi','property.fireplaceExists','property.hasTerrace','transaction.sale.isSubjectToVat','property.energy.hasHeatPump','property.energy.hasPhotovoltaicPanels','property.energy.hasThermicPanels','property.energy.hasCollectiveWaterHeater','property.energy.hasDoubleGlazing','transaction.investor.isInvestmentProperty'
]

cat_feauters=['property.subtype']

ord_features=['property.building.condition','property.kitchen.type','transaction.certificates.epcScore']

imp_features=['flags.isNewlyBuilt','property.bedroomCount','property.bathroomCount','property.netHabitableSurface','property.hasCaretakerOrConcierge','property.hasDisabledAccess','property.hasLift','property.hasLaundryRoom','property.hasGarden','property.parkingCountIndoor','property.parkingCountOutdoor','property.hasSwimmingPool','property.hasSauna','property.hasJacuzzi','property.fireplaceExists','property.hasTerrace','property.energy.hasPhotovoltaicPanels','property.energy.hasDoubleGlazing','property.livingRoom.surface'

]

scale_features=['property.bathroomCount','property.netHabitableSurface','property.livingRoom.surface'
]


df.shape
y=df['transaction.sale.price']
df=df.drop(['transaction.sale.price'],axis=1)


*ORDINAL DATA LEVELS*

In [357]:
#Categories of ordinal values there are null ones will be converted to -1 later will be converted to nan again for imputer 
ord_property_building_condition=['TO_RESTORE','TO_RENOVATE','TO_BE_DONE_UP','GOOD','JUST_RENOVATED','AS_NEW'] 
ord_kitchen_type_ordinal=['NOT_INSTALLED','INSTALLED','USA_INSTALLED', 'SEMI_EQUIPPED', 'USA_SEMI_EQUIPPED', 'HYPER_EQUIPPED', 'USA_HYPER_EQUIPPED']
ord_transaction_certificates_epcScore=['G', 'F', 'E','D', 'C', 'B', 'A', 'A+', 'A++'] 

# from sklearn.preprocessing import OrdinalEncoder
# enc = OrdinalEncoder(categories=[ord_property_building_condition,ord_kitchen_type_ordinal,ord_transaction_certificates_epcScore],handle_unknown='use_encoded_value', unknown_value=-1)
# df[ord_features] = enc.fit_transform(df[ord_features])

#df = df.replace(-1,np.nan)



*Encode Categorical Values*

In [358]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer,IterativeImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder,OrdinalEncoder,LabelEncoder


label_pipeline=[
    ('labelenc',LabelEncoder())
]

cat_pipeline = Pipeline([
    ('onehot', OneHotEncoder())
])
ord_pipeline = Pipeline([
    ('ordencode', OrdinalEncoder(categories=[ord_property_building_condition,ord_kitchen_type_ordinal,ord_transaction_certificates_epcScore],handle_unknown='use_encoded_value', unknown_value=np.nan))
])

imp_pipeline = Pipeline([
    ('impute', IterativeImputer())
])

scale_pipeline = Pipeline([
    ('scale', RobustScaler())
])

preprocess_pipeline = ColumnTransformer([
    
    
    ('cat', cat_pipeline, cat_feauters),
    #('lab',label_pipeline,ord_features),
    ('ord', ord_pipeline, ord_features),
   ('imp', imp_pipeline, imp_features),
    # ('scale', scale_pipeline, scale_features)

    ],remainder='passthrough')  ##remainder is used to get all the columns irrespective of transormation happened or not

preprocess_pipeline2 = ColumnTransformer([
    
      
    #('lab',label_pipeline,ord_features),
    
   ('imp', imp_pipeline, imp_features),
    # ('scale', scale_pipeline, scale_features)

    ],remainder='passthrough')






In [359]:
X = preprocess_pipeline.fit_transform(df)

#df3=preprocess_pipeline2.fit_transform(df_processed)



In [360]:
ii_imp = IterativeImputer()
a=ii_imp.fit_transform(X)
X=a

In [361]:
a=pd.DataFrame(X)
a.to_csv('qq.csv')

In [ ]:
# df=pd.read_csv('qq.csv')

In [ ]:
#????one hot encoder will be added to pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
column_trans=make_column_transformer(
    (OneHotEncoder(), cat_feauters),remainder="passthrough")

df=pd.get_dummies(df, columns=cat_feauters)



Check for nan values

In [ ]:
features_with_na=[features for features in df.columns if df[features].isnull().sum()>1]

null_dict={}
for feature in features_with_na:
    null_dict[feature]=np.round(df[feature].isnull().mean(), 4)




Checkfor unique values of object data types to use in ordinal encoder

In [ ]:
from sklearn import preprocessing
from sklearn.experimental import enable_iterative_imputer

from sklearn.impute import IterativeImputer,KNNImputer
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor

df2=df.copy(deep=True)
df=df.drop(['16'],axis=1)
df2.shape

ii_imp = IterativeImputer()
a=ii_imp.fit_transform(df)

np.around(a,0)

a = a.astype(int)



In [ ]:
#16  y
df1=pd.DataFrame(df_processed)
df1.to_csv('qq.csv')

In [ ]:
print(df.shape)
print(a.shape)

df.loc[:,: ] = a


In [ ]:
y=df['transaction.sale.price'].values
X=df.drop(['transaction.sale.price'],axis=1)

Check for correlations

In [ ]:
plt.subplots(figsize=(16,10))
#sns.heatmap(df.corr())
df.corr().style.background_gradient(cmap="Blues")

# df.info()

In [ ]:
# plt.subplots(figsize=(16,10))
# df.drop("transaction.sale.price",1).corrwith(df['transaction.sale.price']).plot(kind='bar')

Create the model 
Best model is choosen with pycaret

In [370]:
from sklearn.model_selection import ParameterGrid, train_test_split,cross_val_score,cross_validate
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_selector, make_column_transformer,ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

#regr=GradientBoostingRegressor(n_estimators=2000,max_depth=2,min_samples_split=2,learning_rate=0.1)
#regr=RandomForestRegressor(max_depth=50, random_state=0)

regr=CatBoostRegressor(nan_mode= 'Min', eval_metric= 'RMSE', iterations=1000, sampling_frequency= 'PerTree', leaf_estimation_method= 'Newton', grow_policy= 'SymmetricTree', 
                penalties_coefficient=1, boosting_type= 'Plain', model_shrink_mode= 'Constant', feature_border_type= 'GreedyLogSum', l2_leaf_reg=3, random_strength=1, rsm=1, 
                boost_from_average= True, model_size_reg=0.5, subsample=0.800000011920928, use_best_model= False, random_seed=10, depth=6, posterior_sampling= False, border_count=254, 
                 sparse_features_conflict_fraction=0, leaf_estimation_backtracking= 'AnyImprovement', best_model_min_trees=1, model_shrink_rate=0, min_data_in_leaf=1, 
                 loss_function= 'RMSE', learning_rate=0.0396099984645843, score_function= 'Cosine', task_type= 'CPU', leaf_estimation_iterations=1, bootstrap_type= 'MVS',
                  max_leaves=64
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

regr.fit(X_train, y_train)
print("model score: %.3f" % regr.score(X_test, y_test))


0:	learn: 341531.0551449	total: 7.84ms	remaining: 7.83s
1:	learn: 335422.7807375	total: 12.9ms	remaining: 6.43s
2:	learn: 329735.4975589	total: 17.9ms	remaining: 5.95s
3:	learn: 324229.1255618	total: 23.2ms	remaining: 5.78s
4:	learn: 319078.1937822	total: 27.9ms	remaining: 5.55s
5:	learn: 314147.8009159	total: 33.2ms	remaining: 5.5s
6:	learn: 309501.7646872	total: 41.1ms	remaining: 5.83s
7:	learn: 304792.9269121	total: 45.5ms	remaining: 5.64s
8:	learn: 300793.5770075	total: 50.2ms	remaining: 5.52s
9:	learn: 296821.9660215	total: 54.9ms	remaining: 5.43s
10:	learn: 293053.3636343	total: 59.9ms	remaining: 5.38s
11:	learn: 289492.0547500	total: 64.7ms	remaining: 5.33s
12:	learn: 286140.7871168	total: 70ms	remaining: 5.32s
13:	learn: 282808.3251225	total: 77.1ms	remaining: 5.43s
14:	learn: 279847.1325754	total: 81.4ms	remaining: 5.35s
15:	learn: 277028.1874576	total: 86.4ms	remaining: 5.32s
16:	learn: 274267.5848813	total: 91.3ms	remaining: 5.28s
17:	learn: 271583.3429656	total: 95.9ms	rema

In [371]:
print(regr.score(X, y))

0.7436788540390109


In [369]:
# print(regr.score(X_test, y_test))
# regr.best_score_
# a=list(regr.classes_)

# # a=zip(regr.feature_names_, regr.feature_importances_)
# # a=list(a)
# print(a)
from sklearn.metrics import r2_score

y_pred=regr.predict(X_test)
r2_score(y_test, y_pred)


# print(regr.get_feature_importance())
#regr.calc_feature_statistics(X_train,y_train)

0.6343661478295743

Kfold feature selection and other models test

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold,GridSearchCV


#regr = LinearRegression()
#regr=KNeighborsRegressor()
grid=GradientBoostingRegressor(n_estimators=400,max_depth=2,min_samples_split=2,learning_rate=0.1) 

seed = 13
kfold = KFold(n_splits=3, shuffle=True, random_state=seed)

hp_candidates = [{'n_estimators': [400,1000], 'max_depth': [3,16]}]

# Search for best hyperparameters
#grid = GridSearchCV(estimator=regr, param_grid=hp_candidates, cv=2, scoring='r2')
grid.fit(X_train, y_train)




In [ ]:

from sklearn.linear_model import Ridge
ridge = Ridge(alpha = 0.5)
ridge.fit(X_train, y_train)
ridge.score(X_test, y_test)



In [ ]:
from sklearn.linear_model import ElasticNet
en = ElasticNet(alpha = 0.01)
en.fit(X_train, y_train)
en.score(X_test, y_test)

In [ ]:
from sklearn.linear_model import BayesianRidge
bayesian = BayesianRidge()
bayesian.fit(X_train, y_train)
bayesian.score(X_test, y_test)
#bayesian.coef_


In [365]:
from sklearn.linear_model import LinearRegression
ols = LinearRegression()
ols.fit(X_train, y_train)
ols.score(X_test, y_test)



0.4056687052982443